In [1]:
import zipfile

import geopandas as gpd
import pandas as pd
import requests
import shapely

from r5py import TransportNetwork, TravelTimeMatrixComputer, TransportMode

ModuleNotFoundError: No module named 'r5py'

1. Get the extent of AoI

In [7]:
aoi = gpd.read_file("https://github.com/Urban-Analytics-Technology-Platform/demoland-web/raw/main/web/src/data/geography.json")

In [8]:
aoi_poly = aoi.to_crs(27700).unary_union

2. Get H3 grid with the data for the AoI

In [9]:
data_folder = "../../../demoland_data"

Read the full grid

In [10]:
grid = gpd.read_parquet(f"{data_folder}/h3/grid_complete.parquet")

Get a portion of the grid covering AoI.

In [11]:
grid_aoi = grid.iloc[grid.sindex.query(aoi_poly, predicate="intersects")]

3. Make predictive models ready

    - to be done once the final models are settled.

5. Make accessibility ready
    6. Get GTFS
  
Go to https://data.bus-data.dft.gov.uk/downloads/, register and download timetable data for your region in GTFS data format.

In [12]:
gtfs_data_file = f"{data_folder}/raw/accessibility/itm_north_east_gtfs.zip"

7. Get network from OSM

Download a fresh OSM snapshot for England.

In [14]:
r = requests.get('http://download.geofabrik.de/europe/united-kingdom/england-latest.osm.pbf')
with open("england-latest.osm.pbf", "wb") as f:
    f.write(r.content)

Extract the AoI. We need a GeoJSON of the area.

In [15]:
aoi.dissolve().to_file("aoi.geojson")

And then can use osmium to get an extract.

In [16]:
!osmium extract -p aoi.geojson england-latest.osm.pbf -o aoi.osm.pbf

[======================================================================] 100% 


8. Get OS Greenspace

In [30]:
r = requests.get('https://api.os.uk/downloads/v1/products/OpenGreenspace/downloads?area=GB&format=GeoPackage&redirect')
with open("opgrsp_gpkg_gb.zip", "wb") as f:
    f.write(r.content)

Read the file.

In [49]:
with zipfile.ZipFile('opgrsp_gpkg_gb.zip', 'r') as zip_ref:
    with zip_ref.open("Data/opgrsp_gb.gpkg") as gsp:
        f = gsp.read()
        greenspace_sites = gpd.read_file(f, engine="pyogrio", layer="greenspace_site")
        greenspace_access = gpd.read_file(f, engine="pyogrio", layer="access_point")

/home/martin/mambaforge/envs/shap/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: File /vsimem/cdb03436424a4b63a09aff3d6214c59a has GPKG application_id, but non conformant file extension
  result = ogr_read(
/home/martin/mambaforge/envs/shap/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: File /vsimem/bf318594aa5c4178a623728b2ce4d306 has GPKG application_id, but non conformant file extension
  result = ogr_read(


Extract the AoI

In [50]:
greenspace_sites_aoi = greenspace_sites.iloc[greenspace_sites.sindex.query(aoi_poly, predicate="intersects")]
greenspace_access_aoi = greenspace_access.iloc[greenspace_access.sindex.query(aoi_poly, predicate="intersects")]

9. Process OS Greenspace

In [52]:
greenspace_sites_select = greenspace_sites_aoi.query(
    "function!='Allotments Or Community Growing Spaces' & function!='Golf Course' & function!='Bowling Green'"
)
publicpark = greenspace_sites_select.query("function=='Public Park Or Garden'")
playingfield = greenspace_sites_select.query("function=='Playing Field'")
othersport = greenspace_sites_select.query("function=='Other Sports Facility'")
therest = greenspace_sites_select.query(
    "function!='Playing Field' & function!='Public Park Or Garden' & function!='Other Sports Facility'"
)

In [53]:
# find 'therest' not included in the upper categories
# we use sjoin to performe a spatial filter of 'therest' polygons contained in upper categories
join11 = gpd.sjoin(therest, othersport, predicate="within", how="inner")
join12 = gpd.sjoin(therest, playingfield, predicate="within", how="inner")
join13 = gpd.sjoin(therest, publicpark, predicate="within", how="inner")

# generate list of the IDs of 'therest' contained in upper categories, in order to eliminate the corresponding polygons from the layer
list_for_diff11 = join11["id_left"].drop_duplicates().to_list()

diff11 = therest[
    ~therest.id.isin(list_for_diff11)
]  # 1st difference layer # note the negation character ~ to take the polygons NOT included

list_for_diff12 = join12["id_left"].drop_duplicates().to_list()
diff12 = diff11[~diff11.id.isin(list_for_diff12)]  # 2nd difference layer

list_for_diff13 = join13["id_left"].drop_duplicates().to_list()
diff13 = diff12[
    ~diff12.id.isin(list_for_diff13)
]  # 3rd difference layer, this is for 'therest' categories

In [54]:
# we repeat the same operation for subsequent categories:
# find 'othersport' not included in the upper categories
join21 = gpd.sjoin(othersport, playingfield, predicate="within", how="inner")
join22 = gpd.sjoin(othersport, publicpark, predicate="within", how="inner")

list_for_diff21 = join21["id_left"].drop_duplicates().to_list()
diff21 = othersport[~othersport.id.isin(list_for_diff21)]

list_for_diff22 = join22["id_left"].drop_duplicates().to_list()
diff22 = diff21[~diff21.id.isin(list_for_diff22)]  # 'othersport' difference

In [55]:
# find 'playing fields' not included in the upper categories (and viceversa?)
join31 = gpd.sjoin(playingfield, publicpark, predicate="within", how="inner")
join32 = gpd.sjoin(
    publicpark, playingfield, predicate="within", how="inner"
)  ## check it is not empty ... it is empty, we do not use this join

list_for_diff31 = join31["id_left"].drop_duplicates().to_list()
diff31 = playingfield[
    ~playingfield.id.isin(list_for_diff31)
]  # 'playingfield' difference

In [58]:
# put together all the differences layers: (and should bring out to the desired output)
together1 = pd.concat([diff13, diff22]).pipe(
    gpd.GeoDataFrame
)  # 'therest' + 'othersport' differences
together1.head()
together2 = pd.concat([together1, diff31]).pipe(
    gpd.GeoDataFrame
)  # last gdf + 'playingfield' difference
together_again = gpd.GeoDataFrame(pd.concat([together2, publicpark]), crs=27700)  # last gdf + all the public parks)

In [65]:
list_gs_id = together_again["id"].to_list()
accesspoints_edge = greenspace_access_aoi[greenspace_access_aoi.ref_to_greenspace_site.isin(list_gs_id)]
accesspoints_edge = accesspoints_edge.to_crs(27700)

together_again["area_m2"] = together_again["geometry"].area

together_again.to_file("greenspace.gpkg", layer="sites")
accesspoints_edge.to_file("greenspace.gpkg", layer="access_points")

10. Create traveltime matrix (origins are cells, destinations are cells plus greenspace entrances)

In [66]:
grid_aoi

,geometry,air_quality_index,house_price_index,population,"A, B, D, E. Agriculture, energy and water",C. Manufacturing,F. Construction,"G, I. Distribution, hotels and restaurants","H, J. Transport and communication","K, L, M, N. Financial, real estate, professional and administrative activities",...,sisBpM,misCel,ltcRea,ldeAre,lseCCo,lseERI,lteOri,lteWNB,lieWCe,signature_type
hex_id,,,,,,,,,,,,,,,,,,,,,
8919465366bffff,"POLYGON ((433599.645 545332.542, 433457.649 54...",12.235315,1697.775303,2.070927,2.391746,0.397492,0.582602,0.682498,0.338590,0.451670,...,0.013412,16.687437,80.602427,4.246499e+06,0.313751,0.672360,35.187995,0.000736,0.000018,Urban buffer
89194653477ffff,"POLYGON ((436462.663 545552.077, 436320.729 54...",13.027765,1037.946757,23.210263,0.372120,1.113380,1.673051,2.506251,0.967635,1.423365,...,0.056479,11.187832,67.636509,3.930416e+06,0.264439,0.676249,19.150600,0.001114,0.000236,Urban buffer
8919465343bffff,"POLYGON ((436780.718 545576.468, 436638.791 54...",13.027765,1109.766915,20.032634,0.365359,1.100800,1.646476,2.415286,0.953719,1.385548,...,0.047485,33.997918,88.504614,4.013205e+06,0.261993,0.671151,18.673672,0.000832,0.000187,Urban buffer
8919465340fffff,"POLYGON ((436570.397 545847.662, 436428.467 54...",13.027765,670.349332,397.357393,0.387323,1.141585,1.732763,2.711264,0.998887,1.508523,...,0.054465,12.279274,41.338760,1.006992e+06,0.482405,0.871917,33.814698,0.016157,0.000863,Warehouse/Park land
89194653433ffff,"POLYGON ((437098.761 545600.858, 436956.841 54...",12.588189,1280.847121,45.158130,0.279208,0.952710,1.401498,1.681095,0.768209,1.100721,...,0.053967,66.655175,104.635092,3.653489e+06,0.261047,0.698869,23.756565,0.000968,0.000161,Urban buffer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8919733181bffff,"POLYGON ((418844.882 575218.142, 418702.781 57...",10.671450,2552.839899,1.933129,0.113768,0.454039,0.162113,0.713884,0.199639,0.391240,...,0.016849,13.000001,65.388750,4.937058e+06,0.494502,0.960272,0.647615,0.000415,0.030823,Countryside agriculture
89197331813ffff,"POLYGON ((419163.126 575241.929, 419021.031 57...",11.003435,1581.749724,2.889846,0.136670,0.639472,0.439720,0.333585,0.359567,0.376620,...,0.010850,13.000000,77.987134,4.937058e+06,0.494502,0.960272,0.647615,0.000415,0.030823,Urban buffer
891973318c7ffff,"POLYGON ((418952.930 575512.955, 418810.832 57...",11.093738,2679.627966,1.808096,0.110771,0.429800,0.125858,0.763490,0.178746,0.393124,...,0.015533,14.140263,62.976055,4.937058e+06,0.494502,0.960272,0.647615,0.000415,0.030823,Countryside agriculture
